In [69]:
import nibabel as nib
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [70]:
import os


def join_nii_files(file_name_pattern, parent_directory):


    subdirectories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

    individual_data = []

    for subdirectory in subdirectories:
        file_path = os.path.join(parent_directory, subdirectory, file_name_pattern)
        data_raw = nib.load(file_path)
        data = data_raw.get_fdata()
        data_flattened = data.flatten()  #convert 3d-array into a flattened 1d-array
        individual_data.append(data_flattened)
    
    return individual_data

joined_data = np.concatenate((join_nii_files("beta_0002.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Cont"),
                            join_nii_files("beta_0011.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Cont"),
                            join_nii_files("beta_0002.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Pct"),
                            join_nii_files("beta_0011.nii", "C:\\Users\\mathe\\Downloads\\Dados de fMRI\\Pct")), axis=0)


print(joined_data.shape)

brain_valid_data = np.nan_to_num(joined_data, nan=0)   #Replace all the NaN values with 0's

label = np.loadtxt("Data/K-Fold_PCL-5.txt", dtype="float")

(86, 173628)


In [71]:
print(brain_valid_data.shape)

(86, 173628)


In [81]:
def KFoldRegressor_TEPT(X, y):
    
    num_folds = 5
    num_samples = 86

    arr_ord = np.arange(num_samples)
    np.random.shuffle(arr_ord)

    arr_misc = np.concatenate((X[arr_ord[0:17], :], X[arr_ord[17:34], :], X[arr_ord[34:51], :], X[arr_ord[51:68], :], X[arr_ord[68:85], :]), axis = 0)
    label_misc = np.concatenate((y[arr_ord[0:17]], y[arr_ord[17:34]], y[arr_ord[34:51]], y[arr_ord[51:68]], y[arr_ord[68:85]] ), axis = 0)
    
    ind = [0, 17, 34, 51, 68, 85]

    for i in range(5):
        
        rf = RandomForestRegressor(n_estimators = 1)

        slices_deleted = slice(ind[i], ind[i + 1])
        result_arr = np.concatenate((arr_misc[:slices_deleted.start], arr_misc[slices_deleted.stop:]))
        result_label = np.concatenate((label_misc[:slices_deleted.start], label_misc[slices_deleted.stop:]))


        rf.fit(result_arr, result_label)
        y_pred = rf.predict(arr_misc[ind[i]:ind[i+1]])

        print(y_pred)
        print(label_misc[ind[i]:ind[i+1]])





    return

KFoldRegressor_TEPT(brain_valid_data, label)


[52. 58. 53. 52. 52. 58. 41. 34.  3. 15. 63. 17. 15.  7. 15. 32. 17.]
[ 6. 33.  7. 57. 11. 63.  7.  3.  2.  1. 15.  1. 43.  2.  3.  6. 22.]
[22. 53. 17. 43. 22.  7. 22. 33. 52. 32. 53. 17. 17. 33. 15.  3.  0.]
[ 1. 32.  1.  5. 34.  0.  5.  6. 44. 58.  1. 22. 17.  3. 15. 47. 53.]
[ 0. 41. 52. 43. 40.  0. 53. 17. 41.  6. 40.  3.  0. 40. 32. 53. 14.]
[17.  3. 58. 43. 40. 35. 14. 41. 33. 33.  5. 33. 52. 53.  3. 68.  6.]
[33. 41. 68.  1. 34. 57.  1.  3. 43.  7.  1.  3. 43.  5.  2.  3.  7.]
[53.  0.  0.  1. 68. 52. 14. 44. 64.  0. 35.  5. 64. 40. 52. 15.  7.]
[63.  6. 68. 47.  0. 44. 34. 40. 40.  5. 33. 63.  0. 15.  5. 34.  5.]
[58.  0.  0. 47.  3. 57. 34.  7. 58.  3. 11. 63. 41. 32. 53. 15. 52.]
